In [ ]:
from google.colab import drive
import os
import pandas as pd
from openai import OpenAI
import time
import json
import re

In [ ]:
# 1. Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# 2. Path file metadata dan folder CSV serta output
base_folder = "/content/drive/My Drive/AYOK BIKIN DATASET"
csv_folder = os.path.join(base_folder, "[01] TABLE")
metadata_file = os.path.join(base_folder, "[00] METADATA/Metadata Tabel.xlsx")
output_folder = os.path.join(base_folder, "[03] DESCRIPTION")
os.makedirs(output_folder, exist_ok=True)  # Pastikan folder output ada

# Folder input dan output
folder_deskripsi = "/content/drive/My Drive/AYOK BIKIN DATASET/[03] DESCRIPTION"
folder_hasil = "/content/drive/My Drive/AYOK BIKIN DATASET/[04] PARAPHRASE DESCRIPTION [FIX]"
os.makedirs(folder_hasil, exist_ok=True)

Mounted at /content/drive


GPT 4o-mini

In [ ]:
!pip install openai

In [ ]:
# Set your OpenAI API key as an environment variable named OPENAI_API_KEY
# Replace 'YOUR_ACTUAL_API_KEY' with your actual API key
os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'

# Now you can access it using os.environ['OPENAI_API_KEY']
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # Accessing the environment variable
)

In [ ]:
import os
print(os.environ.get('OPENAI_API_KEY'))

In [ ]:
def generate_parafrase(teks, batch_ke):
    """
    Fungsi untuk meminta 10 parafrase per batch.
    Jika batch kedua, berikan instruksi agar tidak mengulang batch pertama tanpa mencantumkan daftar sebelumnya.
    """
    tambahan_instruksi = ""

    if batch_ke == 2:
        tambahan_instruksi = (
            "\n\nPerhatian: Jangan mengulang struktur atau pola kalimat dari parafrase yang telah dihasilkan sebelumnya. "
        )

    prompt = (
        f"Saya ingin anda melakukan parafrase tepat 10 kali terhadap teks yang saya input, tidak lebih dan tidak kurang. "
        "Tujuannya adalah menghasilkan narasi deskriptif dengan variasi linguistik melalui sinonim dan/atau perubahan struktur kalimat. "
        "Saya ingin anda berperan sebagai asisten pemrograman saya untuk otomatisasi parafrase sehingga saya tidak perlu membuat program sendiri. "
        "Penamaan variabel harus sama dengan teks asli. "
        "Khusus untuk kalimat akhir yang sebagai data keseluruhan, jangan tambahkan kata 'total' lagi karena telah diwakilkan oleh wilayahnya. "
        "Gunakan Bahasa Indonesia yang formal, efektif, dan kalimat sesuai EYD. "
        "Jangan menambahkan atau mencampur informasi baru kecuali saya memintanya. "
        f"Teks asli:\n{teks}\n\n"
        f"Kembalikan output dalam format JSON array, seperti ini:\n\n"
        "[\"Parafrase 1\", \"Parafrase 2\", ..., \"Parafrase 10\"]"
        f"{tambahan_instruksi}"  # **Tambahkan instruksi tambahan hanya untuk batch kedua**
    )

    # Panggil API OpenAI
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Anda adalah asisten AI yang membantu parafrase teks."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.8,  # Meningkatkan kreativitas agar lebih bervariasi
    )

    # Ambil output mentah dari OpenAI
    response_text = response.choices[0].message.content.strip()
    print(f"Raw Response (Batch {batch_ke}):", response_text)  # Debugging untuk memastikan JSON valid

    # **Membersihkan format JSON dari karakter yang tidak diperlukan**
    clean_text = re.sub(r'```json|```', '', response_text).strip()  # Hilangkan backticks jika ada

    # Parsing JSON dengan validasi
    try:
        parafrase_list = json.loads(clean_text)

        # Pastikan jumlah sesuai dengan yang diminta
        if not isinstance(parafrase_list, list) or len(parafrase_list) != 10:
            raise ValueError(f"Model tidak mengembalikan tepat 10 elemen di batch {batch_ke}.")

    except (json.JSONDecodeError, ValueError) as e:
        print(f"❌ Gagal membaca JSON dari hasil model di batch {batch_ke}:", e)
        parafrase_list = []

    return parafrase_list  # Mengembalikan daftar hasil parafrase jika valid

In [ ]:
def buat_parafrase(teks):
    """
    Fungsi ini menjalankan prompt dua kali:
    - Pertama untuk menghasilkan 10 parafrase.
    - Kedua untuk menghasilkan 10 parafrase lainnya yang berbeda dari batch pertama.
    Total output yang dihasilkan adalah 20 parafrase.
    """

    # **Menjalankan prompt pertama untuk 10 parafrase pertama**
    hasil_pertama = generate_parafrase(teks, batch_ke=1)

    # **Menjalankan prompt kedua untuk 10 parafrase berikutnya, menghindari duplikasi**
    hasil_kedua = generate_parafrase(teks, batch_ke=2)

    # **Gabungkan hasil pertama dan kedua**
    final_parafrase = hasil_pertama + hasil_kedua

    return final_parafrase  # Mengembalikan total 20 parafrase

In [ ]:
# File untuk mencatat batch yang selesai
processed_batches_file = "/content/drive/My Drive/AYOK BIKIN DATASET/processed_batches.txt"

# Ambil semua file deskripsi
file_deskripsi_list = sorted([f for f in os.listdir(folder_deskripsi) if f.endswith(".txt")])

# **Tentukan ukuran batch**
batch_size = 10  # Setiap batch memproses 10 file

# **Cek batch terakhir yang sudah selesai**
try:
    with open(processed_batches_file, "r") as f:
        processed_batches = set(int(line.strip()) for line in f.readlines())  # Batch yang sudah selesai
except FileNotFoundError:
    processed_batches = set()

# Fungsi untuk memeriksa kelengkapan file
def cek_kelengkapan_file(file_id):
    base_filename = f"D{str(file_id).zfill(4)}"  # Format file output
    parafrase_files = [f for f in os.listdir(folder_hasil) if f.startswith(base_filename)]
    return len(parafrase_files) == 20  # Return True jika file lengkap

# Loop untuk setiap batch
for i in range(0, len(file_deskripsi_list), batch_size):
    batch_number = i // batch_size + 1  # Nomor batch

    # **Lewati batch yang sudah selesai**
    if batch_number in processed_batches:
        print(f"⏩ Melewati Batch {batch_number}, sudah diproses sebelumnya.")
        continue

    batch_files = file_deskripsi_list[i:i + batch_size]  # Ambil 10 file dalam batch ini
    print(f"🚀 Memproses Batch {batch_number}: {len(batch_files)} file...")

    for file_deskripsi in batch_files:
        file_id = int(file_deskripsi.replace("T", "").replace(".txt", ""))  # Ambil ID file
        path_deskripsi = os.path.join(folder_deskripsi, file_deskripsi)

        # **Cek apakah file sudah lengkap**
        if cek_kelengkapan_file(file_id):
            print(f"✔️ File {file_deskripsi} sudah lengkap. Lewati.")
            continue

        # Jika file belum lengkap, baca teks asli
        with open(path_deskripsi, "r", encoding="utf-8") as f:
            deskripsi_asli = f.read().strip()

        # Generate parafrase (20 kali)
        parafrase_hasil = buat_parafrase(deskripsi_asli)

        # Pastikan hasil valid sebelum menyimpan
        if len(parafrase_hasil) < 20:
            print(f"⚠️ Hanya {len(parafrase_hasil)} parafrase yang dihasilkan untuk {file_deskripsi}. Lewati penyimpanan.")
            continue

        # Simpan setiap parafrase dalam file txt terpisah
        base_filename = file_deskripsi.replace("T", "D").replace(".txt", "")

        for idx, teks_parafrase in enumerate(parafrase_hasil, start=1):
            nama_file_hasil = f"{base_filename}_{idx}.txt"
            path_hasil = os.path.join(folder_hasil, nama_file_hasil)

            with open(path_hasil, "w", encoding="utf-8") as f_hasil:
                f_hasil.write(teks_parafrase)

            print(f"✅ Menyimpan file: {path_hasil}")

    # **Catat batch yang sudah selesai agar tidak diulang**
    with open(processed_batches_file, "a") as f:
        f.write(f"{batch_number}\n")

    print(f"✔️ Batch {batch_number} selesai. Menunggu 5 detik sebelum batch berikutnya...")
    time.sleep(5)  # Tambahkan delay untuk menghindari rate limit API


⏩ Melewati Batch 1, sudah diproses sebelumnya.
⏩ Melewati Batch 2, sudah diproses sebelumnya.
⏩ Melewati Batch 3, sudah diproses sebelumnya.
⏩ Melewati Batch 4, sudah diproses sebelumnya.
🚀 Memproses Batch 5: 10 file...
✔️ File T0891.txt sudah lengkap. Lewati.
✔️ File T0892.txt sudah lengkap. Lewati.
✔️ File T0893.txt sudah lengkap. Lewati.
✔️ File T0894.txt sudah lengkap. Lewati.
✔️ File T0895.txt sudah lengkap. Lewati.
✔️ File T0896.txt sudah lengkap. Lewati.
✔️ File T0897.txt sudah lengkap. Lewati.
Raw Response (Batch 1): ```json
[
    "Pada tahun 2015, jumlah siswa Madrasah Aliyah (MA) di Kecamatan Kedungkandang tercatat sebanyak 366 orang, di Kecamatan Sukun sebanyak 318 orang, di Kecamatan Klojen sebanyak 957 orang, di Kecamatan Blimbing sebanyak 23 orang, dan di Kecamatan Lowokwaru mencapai 1.226 orang. Dengan demikian, jumlah siswa Madrasah Aliyah (MA) di Kota Malang berjumlah 2.890 orang.",
    
    "Di tahun 2015, jumlah peserta didik Madrasah Aliyah (MA) di Kecamatan Kedungk

KeyboardInterrupt: 

In [ ]:
def generate_parafrase(teks, batch_ke):
    """
    Fungsi untuk meminta parafrase dalam batch:
    - Batch pertama: 5 parafrase
    - Batch kedua: 5 parafrase
    - Batch ketiga: 5 parafrase
    - Batch keempat: 5 parafrase

    Instruksi tambahan diberikan untuk batch kedua, ketiga, dan keempat agar tidak mengulang hasil sebelumnya.
    """
    tambahan_instruksi = ""

    if batch_ke > 1:
        tambahan_instruksi = (
            "\n\nPerhatian: Jangan mengulang struktur atau pola kalimat dari parafrase yang telah dihasilkan sebelumnya. "
        )

    jumlah_parafrase = 5

    prompt = (
        f"Saya ingin anda melakukan parafrase tepat {jumlah_parafrase} kali terhadap teks yang saya input, tidak lebih dan tidak kurang. "
        "Tujuannya adalah menghasilkan narasi deskriptif dengan variasi linguistik melalui sinonim dan/atau perubahan struktur kalimat. "
        "Saya ingin anda berperan sebagai asisten pemrograman saya untuk otomatisasi parafrase sehingga saya tidak perlu membuat program sendiri. "
        "Penamaan variabel harus sama dengan teks asli. "
        "Khusus untuk kalimat akhir yang sebagai data keseluruhan, jangan tambahkan kata 'total' lagi karena telah diwakilkan oleh wilayahnya. "
        "Gunakan Bahasa Indonesia yang formal, efektif, dan kalimat sesuai EYD. "
        "Jangan menambahkan atau mencampur informasi baru kecuali saya memintanya. "
        f"Teks asli:\n{teks}\n\n"
        f"Kembalikan output dalam format JSON array, seperti ini:\n\n"
        "[\"Parafrase 1\", \"Parafrase 2\", ..., \"Parafrase N\"]"
        f"{tambahan_instruksi}"
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Anda adalah asisten AI yang membantu parafrase teks."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.8,
    )

    response_text = response.choices[0].message.content.strip()
    print(f"Raw Response (Batch {batch_ke}):", response_text)  # Debugging

    clean_text = re.sub(r'```json|```', '', response_text).strip()

    try:
        parafrase_list = json.loads(clean_text)

        if not isinstance(parafrase_list, list) or len(parafrase_list) != jumlah_parafrase:
            raise ValueError(f"Model tidak mengembalikan tepat {jumlah_parafrase} elemen di batch {batch_ke}.")
    except (json.JSONDecodeError, ValueError) as e:
        print(f"❌ Gagal membaca JSON dari hasil model di batch {batch_ke}:", e)
        parafrase_list = []

    return parafrase_list

In [ ]:
def buat_parafrase(teks):
    """
    Fungsi ini menjalankan prompt empat kali:
    - Batch 1 menghasilkan 5 parafrase.
    - Batch 2 menghasilkan 5 parafrase.
    - Batch 3 menghasilkan 5 parafrase.
    - Batch 4 menghasilkan 5 parafrase.
    Total output yang dihasilkan adalah 20 parafrase.
    """
    hasil_pertama = generate_parafrase(teks, batch_ke=1)
    hasil_kedua = generate_parafrase(teks, batch_ke=2)
    hasil_ketiga = generate_parafrase(teks, batch_ke=3)
    hasil_keempat = generate_parafrase(teks, batch_ke=4)

    final_parafrase = hasil_pertama + hasil_kedua + hasil_ketiga + hasil_keempat
    return final_parafrase

In [ ]:
# File untuk mencatat batch yang selesai
processed_batches_file = "/content/drive/My Drive/AYOK BIKIN DATASET/processed_batches.txt"

# Ambil semua file deskripsi
file_deskripsi_list = sorted([f for f in os.listdir(folder_deskripsi) if f.endswith(".txt")])

batch_size = 10  # Setiap batch memproses 10 file

try:
    with open(processed_batches_file, "r") as f:
        processed_batches = set(int(line.strip()) for line in f.readlines())
except FileNotFoundError:
    processed_batches = set()

def cek_kelengkapan_file(file_id):
    base_filename = f"D{str(file_id).zfill(4)}"
    parafrase_files = [f for f in os.listdir(folder_hasil) if f.startswith(base_filename)]
    return len(parafrase_files) == 20

for i in range(0, len(file_deskripsi_list), batch_size):
    batch_number = i // batch_size + 1

    if batch_number in processed_batches:
        print(f"⏩ Melewati Batch {batch_number}, sudah diproses sebelumnya.")
        continue

    batch_files = file_deskripsi_list[i:i + batch_size]
    print(f"🚀 Memproses Batch {batch_number}: {len(batch_files)} file...")

    for file_deskripsi in batch_files:
        file_id = int(file_deskripsi.replace("T", "").replace(".txt", ""))
        path_deskripsi = os.path.join(folder_deskripsi, file_deskripsi)

        if cek_kelengkapan_file(file_id):
            print(f"✔️ File {file_deskripsi} sudah lengkap. Lewati.")
            continue

        with open(path_deskripsi, "r", encoding="utf-8") as f:
            deskripsi_asli = f.read().strip()

        parafrase_hasil = buat_parafrase(deskripsi_asli)

        if len(parafrase_hasil) < 20:
            print(f"⚠️ Hanya {len(parafrase_hasil)} parafrase yang dihasilkan untuk {file_deskripsi}. Lewati penyimpanan.")
            continue

        base_filename = file_deskripsi.replace("T", "D").replace(".txt", "")

        for idx, teks_parafrase in enumerate(parafrase_hasil, start=1):
            nama_file_hasil = f"{base_filename}_{idx}.txt"
            path_hasil = os.path.join(folder_hasil, nama_file_hasil)

            with open(path_hasil, "w", encoding="utf-8") as f_hasil:
                f_hasil.write(teks_parafrase)

            print(f"✅ Menyimpan file: {path_hasil}")

    with open(processed_batches_file, "a") as f:
        f.write(f"{batch_number}\n")

    print(f"✔️ Batch {batch_number} selesai. Menunggu 5 detik sebelum batch berikutnya...")
    time.sleep(5)

⏩ Melewati Batch 1, sudah diproses sebelumnya.
🚀 Memproses Batch 2: 10 file...
✔️ File T0861.txt sudah lengkap. Lewati.
✔️ File T0862.txt sudah lengkap. Lewati.
✔️ File T0863.txt sudah lengkap. Lewati.
✔️ File T0864.txt sudah lengkap. Lewati.
✔️ File T0865.txt sudah lengkap. Lewati.
✔️ File T0866.txt sudah lengkap. Lewati.
✔️ File T0867.txt sudah lengkap. Lewati.
✔️ File T0868.txt sudah lengkap. Lewati.
✔️ File T0869.txt sudah lengkap. Lewati.
Raw Response (Batch 1): ```json
[
    "Pada tahun 2019, jumlah penduduk (de jure) berusia 0-18 tahun yang memiliki akta kelahiran di Kecamatan Kedungkandang tercatat sebanyak 51.683 jiwa, di Kecamatan Sukun sebanyak 25.600 jiwa, di Kecamatan Klojen sebanyak 58.157 jiwa, di Kecamatan Blimbing sebanyak 55.403 jiwa, dan di Kecamatan Lowokwaru sebanyak 43.867 jiwa. Dengan demikian, jumlah penduduk (de jure) berusia 0-18 tahun yang memiliki akta kelahiran di Kota Malang adalah 234.710 jiwa.",
    
    "Di tahun 2019, jumlah penduduk (de jure) yang ber

KeyboardInterrupt: 

In [ ]:
import random

# Path folder
folder_metadata = "/content/drive/My Drive/AYOK BIKIN DATASET/[00] METADATA"
folder_deskripsi = "/content/drive/My Drive/AYOK BIKIN DATASET/[03] DESCRIPTION"
folder_hasil = "/content/drive/My Drive/AYOK BIKIN DATASET/[05] TEST PARAPHRASE DESCRIPTION"

# Path metadata file
metadata_file = os.path.join(folder_metadata, "Metadata Tabel.xlsx")

# Output file untuk human evaluation
output_file = "/content/drive/My Drive/AYOK BIKIN DATASET/human_evaluation.xlsx"

# Ambil metadata
metadata_df = pd.read_excel(metadata_file)
metadata_df['Table ID'] = metadata_df['Table ID'].astype(str)

# Ambil daftar semua file deskripsi dan parafrase
file_deskripsi_list = sorted([f for f in os.listdir(folder_deskripsi) if f.endswith(".txt")])
file_parafrase_list = sorted([f for f in os.listdir(folder_hasil) if f.endswith(".txt")])

# Pilih 240 file parafrase secara acak
random_files = random.sample(file_parafrase_list, 240)

# Proses file untuk human evaluation
data = []
for parafrase_file in random_files:
    # Ambil ID tabel dari nama file
    id_tabel = parafrase_file.split("_")[0].replace("D", "T")  # Sesuaikan ID tabel ke format metadata

    # Ambil deskripsi base
    deskripsi_file = f"{id_tabel}.txt"
    path_deskripsi = os.path.join(folder_deskripsi, deskripsi_file)
    with open(path_deskripsi, "r", encoding="utf-8") as f:
        deskripsi_base = f.read().strip()

    # Ambil hasil parafrase
    path_parafrase = os.path.join(folder_hasil, parafrase_file)
    with open(path_parafrase, "r", encoding="utf-8") as f:
        hasil_parafrase = f.read().strip()

    # Tambahkan data ke list
    data.append({
        "Table ID": id_tabel,
        "Deskripsi Base": deskripsi_base,
        "Hasil Parafrase": hasil_parafrase,
        "Fluency": "",  # Kosong untuk diisi manual
        "Faithfulness/Accuracy": "",  # Kosong untuk diisi manual
        "Relevancy": ""  # Kosong untuk diisi manual
    })

# Simpan ke Excel
df = pd.DataFrame(data)
df.to_excel(output_file, index=False)

print(f"Human evaluation file saved to {output_file}")

In [ ]:
# Evaluasi Otomatis
!pip install -q rouge-score bert-score nltk scikit-learn
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import csv

from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Download the 'punkt_tab' data for tokenization
nltk.download('punkt_tab')
nltk.download('punkt')

# Fungsi Evaluasi
def calculate_bleu(reference, candidate):
    reference_tokens = [nltk.word_tokenize(reference)]
    candidate_tokens = nltk.word_tokenize(candidate)
    return sentence_bleu(reference_tokens, candidate_tokens)

def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores['rougeL'].fmeasure

def calculate_bert(reference, candidates):
    P, R, F1 = bert_score(candidates, [reference], lang="id")
    return float(F1.mean())

def calculate_cosine_similarity(reference, candidates):
    vectorizer = TfidfVectorizer().fit_transform([reference] + candidates)
    vectors = vectorizer.toarray()
    reference_vector = vectors[0]
    similarities = cosine_similarity([reference_vector], vectors[1:])
    return similarities.flatten().mean()

# Load IndoBERT tokenizer dan model
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

# Fungsi untuk mendapatkan embedding
def get_embedding(text):
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        output = model(**tokens)
    # Mengambil mean dari last hidden state sebagai representasi vektor
    embedding = output.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

# Fungsi untuk menghitung cosine similarity menggunakan IndoBERT
def calculate_indobert_similarity(reference, candidates):
    ref_embedding = get_embedding(reference)
    cand_embeddings = np.array([get_embedding(cand) for cand in candidates])
    similarities = cosine_similarity([ref_embedding], cand_embeddings)
    return similarities.flatten().mean()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 999.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

In [ ]:
# # Contoh penggunaan
# reference_text = "Pada tahun 2021, Jumlah Penduduk (De Jure) di Kecamatan Kedungkandang sebanyak 225.337 jiwa, di Kecamatan Sukun sebanyak 216.918 jiwa, di Kecamatan Klojen sebanyak 110.796 jiwa, di Kecamatan Blimbing sebanyak 203.380 jiwa, dan di Kecamatan Lowokwaru sebanyak 181.445 jiwa. Oleh karena itu, Jumlah Penduduk (De Jure) di Kota Malang sebanyak 937.876 jiwa."
# candidate_texts = [
#     "Pada tahun 2021, jumlah penduduk (De Jure) di Kecamatan Kedungkandang tercatat sebanyak 225.337 jiwa, di Kecamatan Sukun berjumlah 216.918 jiwa, di Kecamatan Klojen sebanyak 110.796 jiwa, di Kecamatan Blimbing mencapai 203.380 jiwa, dan di Kecamatan Lowokwaru sejumlah 181.445 jiwa. Dengan demikian, jumlah penduduk (De Jure) di Kota Malang adalah 937.876 jiwa.",
#     "Di tahun 2021, penduduk (De Jure) di Kecamatan Kedungkandang mencapai 225.337 jiwa, di Kecamatan Sukun tercatat 216.918 jiwa, di Kecamatan Klojen sebanyak 110.796 jiwa, di Kecamatan Blimbing berjumlah 203.380 jiwa, dan di Kecamatan Lowokwaru sebanyak 181.445 jiwa. Maka, total penduduk (De Jure) di Kota Malang adalah 937.876 jiwa.",
#     "Tahun 2021 mencatat jumlah penduduk (De Jure) di Kecamatan Kedungkandang sebanyak 225.337 jiwa, sedangkan di Kecamatan Sukun sejumlah 216.918 jiwa, di Kecamatan Klojen mencapai 110.796 jiwa, di Kecamatan Blimbing berjumlah 203.380 jiwa, dan di Kecamatan Lowokwaru sebanyak 181.445 jiwa. Oleh karena itu, jumlah penduduk (De Jure) di Kota Malang adalah 937.876 jiwa."
# ]

# similarity_score = calculate_indobert_similarity(reference_text, candidate_texts)
# print(f"IndoBERT Cosine Similarity: {similarity_score:.4f}")

IndoBERT Cosine Similarity: 0.9852


In [ ]:
evaluation_results_path = os.path.join(folder_hasil, "evaluation_results.csv")

# Evaluasi Otomatis
file_deskripsi_list = sorted([f for f in os.listdir(folder_deskripsi) if f.endswith(".txt")])[0:100]

with open(evaluation_results_path, "w", newline="", encoding="utf-8") as eval_file:
    writer = csv.writer(eval_file)
    writer.writerow(["File", "BLEU", "ROUGE-L", "BERTScore", "Cosine Similarity", "IndoBERT Similarity"])

    for file_deskripsi in file_deskripsi_list:
        path_deskripsi = os.path.join(folder_deskripsi, file_deskripsi)
        with open(path_deskripsi, "r", encoding="utf-8") as f:
            deskripsi = f.read().strip()

        # Nama dasar file untuk mencocokkan parafrase
        base_filename = file_deskripsi.replace("T", "D").replace(".txt", "")
        parafrase_files = sorted(
            [f for f in os.listdir(folder_hasil) if f.startswith(base_filename)]
        )

        # Evaluasi setiap parafrase
        bleu_scores, rouge_scores, bert_scores, cosine_scores, indobert_scores = [], [], [], [], []
        for parafrase_file in parafrase_files:
            path_parafrase = os.path.join(folder_hasil, parafrase_file)
            with open(path_parafrase, "r", encoding="utf-8") as f_para:
                parafrase = f_para.read().strip()

            # Hitung metrik evaluasi
            bleu_scores.append(calculate_bleu(deskripsi, parafrase))
            rouge_scores.append(calculate_rouge(deskripsi, parafrase))
            bert_scores.append(calculate_bert(deskripsi, [parafrase]))
            cosine_scores.append(calculate_cosine_similarity(deskripsi, [parafrase]))
            indobert_scores.append(calculate_indobert_similarity(deskripsi, [parafrase]))

        # Rata-rata metrik untuk file ini
        avg_bleu = sum(bleu_scores) / len(bleu_scores)
        avg_rouge = sum(rouge_scores) / len(rouge_scores)
        avg_bert = sum(bert_scores) / len(bert_scores)
        avg_cosine = sum(cosine_scores) / len(cosine_scores)
        avg_indobert = sum(indobert_scores) / len(indobert_scores)

        # Tulis hasil evaluasi ke CSV
        writer.writerow([file_deskripsi, avg_bleu, avg_rouge, avg_bert, avg_cosine, avg_indobert])
        print(f"Evaluasi selesai untuk {file_deskripsi}.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Evaluasi selesai untuk T0024.txt.


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Evaluasi selesai untuk T0025.txt.
Evaluasi selesai untuk T0026.txt.


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Evaluasi selesai untuk T0027.txt.
Evaluasi selesai untuk T0028.txt.
Evaluasi selesai untuk T0029.txt.
Evaluasi selesai untuk T0030.txt.
Evaluasi selesai untuk T0031.txt.
Evaluasi selesai untuk T0032.txt.
Evaluasi selesai untuk T0033.txt.
Evaluasi selesai untuk T0034.txt.
Evaluasi selesai untuk T0035.txt.
Evaluasi selesai untuk T0036.txt.
Evaluasi selesai untuk T0037.txt.
Evaluasi selesai untuk T0038.txt.
Evaluasi selesai untuk T0039.txt.
Evaluasi selesai untuk T0040.txt.
Evaluasi selesai untuk T0041.txt.
Evaluasi selesai untuk T0042.txt.
Evaluasi selesai untuk T0043.txt.
Evaluasi selesai untuk T0044.txt.
Evaluasi selesai untuk T0045.txt.
Evaluasi selesai untuk T0046.txt.
Evaluasi selesai untuk T0047.txt.
Evaluasi selesai untuk T0048.txt.
Evaluasi selesai untuk T0049.txt.
Evaluasi selesai untuk T0050.txt.
Evaluasi selesai untuk T0051.txt.
Evaluasi selesai untuk T0052.txt.
Evaluasi selesai untuk T0053.txt.
Evaluasi selesai untuk T0054.txt.
Evaluasi selesai untuk T0055.txt.
Evaluasi seles